# Libraries

In [1]:
# Import BW25 packages. You'll notice the packages are imported individually, unlike a one-and-done import with BW2.
import bw2data as bd
# import bw2io as bi
import bw2calc as bc
# import bw2analyzer as bwa
# import matplotlib.pyplot as plt
# from collections import defaultdict
import numpy as np
# import seaborn as sns
# import pandas as pd

In [2]:
from bw2analyzer import ContributionAnalysis

In [3]:
# Alias np.NaN to np.nan for backward compatibility
np.NaN = np.nan

# 1. Support Functions

In [4]:
from database_setup import find_activity_by_name_product_location
from database_setup import results_to_dataframe

from lifecycle import calculate_exchange_impacts
from lifecycle import calculate_impacts_for_activities
from lifecycle import run_comprehensive_lcia
from lifecycle import find_most_impactful_exchanges

from config import activities_nmcoxide, reference_product_nmcoxide
from config import recipe_midpoint_h, activities_li, reference_product_lithium, activities_ni, reference_product_nickel, activities_mn, reference_product_manganese

# 2. Imports and Declarations

In [5]:
project_name = "LNV-EI38-202502025"

In [6]:
#Creating/accessing the project
bd.projects.set_current("LNV-EI38-202502025")

In [8]:
list(bd.databases)

['biosphere3',
 'ecoinvent 3.8 cutoff',
 'EI38_cutoff_remind_SSP1-Base_2020_baseline',
 'EI38_cutoff_remind_SSP1-Base_2025_baseline',
 'EI38_cutoff_remind_SSP1-Base_2030_baseline',
 'EI38_cutoff_remind_SSP1-Base_2035_baseline',
 'EI38_cutoff_remind_SSP1-Base_2040_baseline',
 'EI38_cutoff_remind_SSP2-Base_2020_baseline',
 'EI38_cutoff_remind_SSP2-Base_2025_baseline',
 'EI38_cutoff_remind_SSP2-Base_2030_baseline',
 'EI38_cutoff_remind_SSP2-Base_2035_baseline',
 'EI38_cutoff_remind_SSP2-Base_2040_baseline',
 'EI38_cutoff_remind_SSP5-Base_2020_baseline',
 'EI38_cutoff_remind_SSP5-Base_2025_baseline',
 'EI38_cutoff_remind_SSP5-Base_2030_baseline',
 'EI38_cutoff_remind_SSP5-Base_2035_baseline',
 'EI38_cutoff_remind_SSP5-Base_2040_baseline']

In [ ]:
db_name = 'EI38_cutoff_remind_SSP1-Base_2025_baseline'

# 3. Activity and pLCIA

In [ ]:
# Search term from updated SSP1-Base2020 database.
activity_name = 'platinum group metal mine operation, ore with high palladium content'
# Load the database
db_name = 'EI38_cutoff_remind_SSP1-Base_2020_energy'
# Use your function to find the activity
activity = find_activity_by_name_product_location(db_name, activity_name, reference_product_nickel)

In [ ]:
# Search term from updated SSP1-Base2020 database.
activity_name = 'lithium carbonate production, from spodumene'
location = 'RoW'
# Load the database
db_name = 'EI38_cutoff_remind_SSP1-Base_2020_energy'
# Use your function to find the activity
activity = find_activity_by_name_product_location(db_name, activity_name, location=location)

In [ ]:
# Search term from updated SSP1-Base2020 database.
activity_name = 'smelting and refining of nickel concentrate, 7% Ni'
location = 'CN'
# Load the database
db_name = 'EI38_cutoff_remind_SSP1-Base_2020_energy'
# Use your function to find the activity
activity = find_activity_by_name_product_location(db_name, activity_name, location=location)

In [ ]:
# Search term from updated SSP1-Base2020 database.
activity_name = 'nickel mine operation and benefication to nickel concentrate, 16% Ni'
location = 'CA-QC'
# Load the database
db_name = 'EI38_cutoff_remind_SSP1-Base_2025_baseline'
# Use your function to find the activity
activity = find_activity_by_name_product_location(db_name, activity_name, 'nickel concentrate, 16% Ni', location=location)

# 4. Contribution Analysis

## 4.1. Confirming elementary flows

In [ ]:
list(activity.exchanges())

In [ ]:
list(activity.biosphere())

## 4.2. Verifying LCIA

### 4.2.1. Single impact category

In [ ]:
# Define the functional unit (e.g., 1 unit of the activity)
functional_unit = {activity: 1}

# Define the LCIA method you want to use, e.g., ReCiPe Endpoint
method = ('ReCiPe Endpoint (H,A)', 'human health', 'total')

# Create an LCA object
lca = bc.LCA(functional_unit, method)

# Perform the LCI and LCIA
lca.lci()
lca.lcia()
lca.score

### 4.2.2. All impact categories

In [ ]:
'''def run_comprehensive_lcia(activity, methods_list):
    """
    Perform a comprehensive LCIA for a given activity across multiple impact categories.
    
    Parameters:
    - activity: The specific activity for which the LCIA is to be performed.
    - methods_list: A list of tuples representing the impact assessment methods.

    Returns:
    - lca_results: A dictionary with methods as keys and their corresponding LCIA scores as values.
    """

    # Define the functional unit (e.g., 1 unit of the activity)
    functional_unit = {activity: 1}

    # Initialize a dictionary to store the results
    lca_results = defaultdict(float)

    # Loop over all impact categories in the method
    for method in methods_list:
        # Run LCI and LCIA
        lca = bc.LCA(functional_unit, method)
        lca.lci()
        lca.lcia()
        
        # Store the result for each category
        lca_results[method] = lca.score

    # Output the results
    for category, score in lca_results.items():
        print(f"{category}: {score}")
    
    return lca_results'''

In [ ]:
run_comprehensive_lcia(activity, recipe_midpoint_h)

In [ ]:
ca = ContributionAnalysis()

# Get the top processes contributing to the LCIA score
top_processes = ca.annotated_top_processes(lca, limit=10)  # You can adjust the limit as needed
print("Top Processes:")
for process in top_processes:
    print(process)

# 5. Finding most impactful exchanges and exporting - Per activity

In [ ]:
activity

In [ ]:
# Define the functional unit (e.g., 1 unit of the activity)
functional_unit = {activity: 1}

# Define the LCIA method you want to use, e.g., ReCiPe Endpoint
method = ('ReCiPe Endpoint (H,A)', 'human health', 'total')

In [ ]:
'''def find_most_impactful_exchanges(lca, top_n=2):
    """
    Find the most impactful exchanges (both technosphere and biosphere) from a characterized LCA.
    
    Parameters:
    - lca: A Brightway2 LCA object that has already been run.
    - top_n: Number of top contributors to return.
    
    Returns:
    - A list of tuples containing the top N most impactful exchanges and their contributions.
    """
    # Initialize a list to store contributions
    exchange_contributions = []
    
    # Get the reverse dictionaries for technosphere and biosphere
    rev_techno, rev_prod, rev_bio = lca.reverse_dict()

    # Loop over the non-zero elements of the characterized inventory
    for row, col in zip(*lca.characterized_inventory.nonzero()):
        contribution = lca.characterized_inventory[row, col]
        
        # Check if this is a biosphere or technosphere exchange
        if col in rev_bio:
            # Biosphere flow
            flow = rev_bio[col]
            flow_name = bd.get_activity(flow)["name"]
            exchange_contributions.append((contribution, flow_name, "biosphere"))
        else:
            # Technosphere flow (process contribution)
            process = rev_techno[col]
            process_name = bd.get_activity(process)["name"]
            exchange_contributions.append((contribution, process_name, "technosphere"))

    # Sort contributions by absolute impact (descending)
    exchange_contributions.sort(key=lambda x: abs(x[0]), reverse=True)
    
    return exchange_contributions[:top_n]'''

In [ ]:
'''# After running LCI and LCIA
lca = bc.LCA({activity: 1}, method)
lca.lci()
lca.lcia()
'''

# Find the top 10 most impactful exchanges
most_impactful_exchanges = find_most_impactful_exchanges(lca, top_n=10)

# Print the results
for contribution, name, exchange_type in most_impactful_exchanges:
    print(f"Contribution: {contribution:.5f}, Name: {name}, Type: {exchange_type}")

# 6. Output of all impacts, listed by EFs with most contribution

In [ ]:
activities_mining = [
    ('nickel mine operation and benefication to nickel concentrate, 7% Ni','CN')
    #('platinum group metal, extraction and refinery operations', 'ZA')
    # ('platinum group metal, mine and concentration operations', 'ZA')
    # ('processing of nickel-rich materials', 'GLO')
    # ('smelting and refining of nickel concentrate, 16% Ni', 'GLO')
    # ('nickel mine operation and benefication to nickel concentrate, 16% Ni', 'CA-QC')
    # ('cobalt production', 'GLO')
    # ('spodumene production','RoW')
    # ('lithium brine inspissation','GLO')
    # ('lithium carbonate production, from concentrated brine','GLO')
    # ('manganese(III) oxide production','CN')
    # ('manganese sulfate production','GLO')
    # ('manganese concentrate production','GLO')
]

# reference_product_nickel_mining = 'manganese concentrate'

#'platinum group metal concentrate'
# 'nickel, class 1'


In [ ]:
results = calculate_impacts_for_activities(activities_mining, recipe_midpoint_h, db_name, reference_product_nickel)

In [ ]:
results

In [ ]:
# Assuming results, project_name, and db_name are already defined
df = results_to_dataframe(results, project_name, db_name)

In [ ]:
df.to_csv('14_A.csv')

# 7. Calling Ni, Li, and Mn

## 7.1. Nickel

In [ ]:
# Call the function
results = calculate_impacts_for_activities(activities_ni, recipe_midpoint_h, db_name, reference_product_nickel)

In [ ]:
results

In [ ]:
# Assuming results, project_name, and db_name are already defined
df = results_to_dataframe(results, project_name, db_name)

In [ ]:
df.to_csv('./contribution_analysis/EFcontributions_nickel.csv')

## 7.2. Lithium

In [ ]:
# Call the function
results = calculate_impacts_for_activities(activities_li, recipe_midpoint_h, db_name)

In [ ]:
# Assuming results, project_name, and db_name are already defined
df = results_to_dataframe(results, project_name, db_name)

In [ ]:
df.to_csv('./contribution_analysis/EFcontributions_lithium.csv')

## 7.3. Manganese

In [ ]:
# Call the function
results = calculate_impacts_for_activities(activities_mn, recipe_midpoint_h, db_name, reference_product_manganese)

In [ ]:
# Assuming results, project_name, and db_name are already defined
df = results_to_dataframe(results, project_name, db_name)

In [ ]:
df.to_csv('./contribution_analysis/EFcontributions_manganese.csv')

# 8. NMC oxide

In [ ]:
# Call the function
results = calculate_impacts_for_activities(activities_nmcoxide, recipe_midpoint_h, db_name)

In [ ]:
# Assuming results, project_name, and db_name are already defined
df = results_to_dataframe(results, project_name, db_name)

In [ ]:
df.to_csv('./contribution_analysis/EFcontributions_nmcoxide-postVSI2025.csv')

In [ ]:
'''
Running again post permanent modify
'''

# 9 test

In [9]:
db_name = 'ecoinvent 3.8 cutoff'

In [10]:
# Call the function
results = calculate_impacts_for_activities(activities_ni, recipe_midpoint_h, db_name, reference_product_nickel)

Calculating impacts for activity 'treatment of metal part of electronics scrap, in copper, anode, by electrolytic refining' in location 'RoW' using method '('ReCiPe Midpoint (H)', 'terrestrial ecotoxicity', 'TETPinf')'...

Top impacts for activity 'treatment of metal part of electronics scrap, in copper, anode, by electrolytic refining' in location 'RoW' and method '('ReCiPe Midpoint (H)', 'terrestrial ecotoxicity', 'TETPinf')':
Exchange: market for metal part of electronics scrap, in copper, anode, Type: technosphere, Compartment: None, Impact: 5.341118805007399e-05
Exchange: market for sulfuric acid, Type: technosphere, Compartment: None, Impact: 2.8521607589205628e-06
Exchange: market group for electricity, high voltage, Type: technosphere, Compartment: None, Impact: 2.1045474568881506e-06
Exchange: Copper, Type: biosphere, Compartment: ('air', 'non-urban air or from high stacks'), Impact: 2.0973318277366393e-06
Exchange: market group for electricity, high voltage, Type: technospher

KeyboardInterrupt: 

In [ ]:
# Assuming results, project_name, and db_name are already defined
df = results_to_dataframe(results, project_name, db_name)

In [ ]:
df.to_csv('./contribution_analysis/EI38-EFcontributions_nickel.csv')

In [ ]:
# Call the function
results = calculate_impacts_for_activities(activities_li, recipe_midpoint_h, db_name)

In [ ]:
# Assuming results, project_name, and db_name are already defined
df = results_to_dataframe(results, project_name, db_name)

In [ ]:
df.to_csv('./contribution_analysis/EI38-EFcontributions_lithium.csv')

In [ ]:
# Call the function
results = calculate_impacts_for_activities(activities_mn, recipe_midpoint_h, db_name, reference_product_manganese)

In [ ]:
# Assuming results, project_name, and db_name are already defined
df = results_to_dataframe(results, project_name, db_name)

In [ ]:
df.to_csv('./contribution_analysis/EFcontributions_manganese.csv')

In [ ]:
# Call the function
results = calculate_impacts_for_activities(activities_nmcoxide, recipe_midpoint_h, db_name)

In [ ]:
# Assuming results, project_name, and db_name are already defined
df = results_to_dataframe(results, project_name, db_name)

In [ ]:
df.to_csv('./contribution_analysis/EI38-Fcontributions_nmcoxide.csv')